In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.utils import np_utils
import numpy as np
import cv2
import time
import os
import matplotlib.pyplot as plt
import random
np.random.seed(10)

In [ ]:
################################################load data and process################################################

In [ ]:
#padding with 0 to get 1:1 size ratio
def img_padding(img):
    x, y = img.shape
    size_diff = abs(x-y)
    pad_size = int(size_diff/2)
    if(x>y):
        pad = np.zeros((x, pad_size),dtype=int)
        img_1_1 = np.concatenate((pad,img,pad), axis = 1)
        if(size_diff%2 != 0):
            pad1 = np.zeros((x, 1),dtype=int)
            img_1_1 = np.concatenate((img_1_1,pad1), axis = 1)
    elif(y>x):
        pad = np.zeros((pad_size, y),dtype=int)
        img_1_1 = np.concatenate((pad,img,pad), axis = 0)
        if(size_diff%2 != 0):
            pad1 = np.zeros((1, y),dtype=int)
            img_1_1 = np.concatenate((img_1_1,pad1), axis = 0)
    else:
        img_1_1 = img

    return img_1_1

In [ ]:
# this function is for read image,the input is directory name (for RGB)    
def read_directory(directory_name):
    array_of_img = [] # this if for store all of the image data
    dim = (224,224)
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #print(filename) #just for test
        #img is used to store the image data 
        img = cv2.imread(directory_name + "/" + filename, cv2.IMREAD_GRAYSCALE)
        img = img_padding(img)
        img = img.astype(np.uint8)
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        array_of_img.append(img)
    return array_of_img

In [ ]:
#read feature
truck_X = read_directory("training_imgs/RGB_imgs/cap")
truck_X = np.asarray(truck_X)
truck_X = truck_X.reshape((len(truck_X), 224, 224, 1))
forklift_X = read_directory("training_imgs/RGB_imgs/flashlight")
forklift_X = np.asarray(forklift_X)
forklift_X = forklift_X.reshape((len(forklift_X), 224, 224, 1))
negative_X = read_directory("training_imgs/RGB_imgs/soda_can")
negative_X = np.asarray(negative_X)
negative_X = negative_X.reshape((len(negative_X), 224, 224, 1))

print(truck_X.shape)
print(forklift_X.shape) 
print(negative_X.shape)

print(type(negative_X))

In [ ]:
#############################################feature################################################

In [ ]:
#traning and testing feature
#you can choose the training and testing data by this (by assign)
#or use the "train_test_split" function from sklearn.model as down below
truck_X_train = truck_X[-100:]
truck_X_test = truck_X[0:100]
forklift_X_train = forklift_X[-100:]
forklift_X_test = forklift_X[0:100]
negative_X_train = negative_X[-100:]
negative_X_test = negative_X[0:100]

print(truck_X_train.shape, truck_X_test.shape)
print(forklift_X_train.shape, forklift_X_test.shape)
print(negative_X_train.shape, negative_X_test.shape)

In [ ]:
#concatenate 4 categories into training and testing feature
train_X = np.concatenate(( truck_X_train, forklift_X_train, negative_X_train), axis=0)
test_X = np.concatenate(( truck_X_test, forklift_X_test, negative_X_test), axis=0)
print(train_X.shape, test_X.shape)

In [ ]:
#############################################label################################################

In [ ]:
#training and testing label
truck_Y_train = np.ones(truck_X_train.shape[0])*0
truck_Y_test = np.ones(truck_X_test.shape[0])*0

forklift_Y_train = np.ones(forklift_X_train.shape[0])*1
forklift_Y_test = np.ones(forklift_X_test.shape[0])*1

negative_Y_train = np.ones(negative_X_train.shape[0])*2
negative_Y_test = np.ones(negative_X_test.shape[0])*2


print(truck_Y_train.shape, truck_Y_test.shape)
print(forklift_Y_train.shape, forklift_Y_test.shape)
print(negative_Y_train.shape, negative_Y_test.shape)

In [ ]:
#concatenate 4 categories into training and testing label
train_Y = np.concatenate(( truck_Y_train, forklift_Y_train, negative_Y_train), axis=0)
test_Y = np.concatenate(( truck_Y_test, forklift_Y_test, negative_Y_test), axis=0)
print(train_Y.shape, test_Y.shape)

In [ ]:
#############################################shuffle################################################

In [ ]:
#shuffle the training feature and label
temp = list(zip(train_X, train_Y))
random.shuffle(temp)
train_X, train_Y = zip(*temp)
train_X = np.asarray(train_X)
train_Y = np.asarray(train_Y)
print(train_X.shape, train_Y.shape)

In [ ]:
#shuffle the testing feature and label
temp = list(zip(test_X, test_Y))
random.shuffle(temp)
test_X, test_Y = zip(*temp)
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)
print(test_X.shape, test_Y.shape)

In [ ]:
#reform training and testing label into one hot format
train_Y_onehot = np_utils.to_categorical(train_Y)
test_Y_onehot = np_utils.to_categorical(test_Y)
print(train_Y_onehot.shape, test_Y_onehot.shape)

In [ ]:
#############################################split to get validation data################################################

In [ ]:
from sklearn.model_selection import train_test_split
train_images, validation_images, train_labels, validation_labels = train_test_split(train_X, train_Y_onehot, test_size=0.2, random_state=42)

In [ ]:
print('train imgs:', train_images.shape, 'labels:', train_labels.shape)
print('validation imgs:', validation_images.shape, 'labels:', validation_labels.shape)
print('test imgs:', test_X.shape, 'labels:', test_Y_onehot.shape)

In [ ]:
#############################################define model funtions, dense, conv2d and mas_pooling#############################################

In [ ]:
def layer(output_dim, input_dim, inputs, activation=None):  #dense for MLP
  W = tf.Variable(tf.random_normal([input_dim, output_dim]))
  b = tf.Variable(tf.random_normal([1, output_dim]))
  XWb = tf.matmul(inputs, W) + b
  if activation is None:
    outputs = XWb
  else:
    outputs = activation(XWb)
  return outputs

In [ ]:
def dropout(h):  #dropout
  return tf.nn.dropout(h, rate=0.2)

In [ ]:
def weight(shape):  #conv2d
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias(shape):
  return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W, b):
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')+b 

In [ ]:
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
#############################################model training#############################################

In [ ]:
#build CNN model
x = tf.placeholder("float", [None, 224, 224, 1])
#conv block 1
W1_1 = weight([3,3,1,64])
b1_1 = bias([64])
conv1_1 = tf.nn.relu(conv2d(x, W1_1, b1_1))

W1_2 = weight([3,3,64,64])
b1_2 = bias([64])
conv1_2 = tf.nn.relu(conv2d(conv1_1, W1_2, b1_2))

C1_pool = max_pool_2x2(conv1_2)

In [ ]:
#conv block 2
W2_1 = weight([3,3,64,128])
b2_1 = bias([128])
conv2_1 = tf.nn.relu(conv2d(C1_pool, W2_1, b2_1))

W2_2 = weight([3,3,128,128])
b2_2 = bias([128])
conv2_2 = tf.nn.relu(conv2d(conv2_1, W2_2, b2_2))

C2_pool = max_pool_2x2(conv2_2)

In [ ]:
#conv block 3
W3_1 = weight([3,3,128,256])
b3_1 = bias([256])
conv3_1 = tf.nn.relu(conv2d(C2_pool, W3_1, b3_1))

W3_2 = weight([3,3,256,256])
b3_2 = bias([256])
conv3_2 = tf.nn.relu(conv2d(conv3_1, W3_2, b3_2))

W3_3 = weight([3,3,256,256])
b3_3 = bias([256])
conv3_3 = tf.nn.relu(conv2d(conv3_2, W3_3, b3_3))

C3_pool = max_pool_2x2(conv3_3)

In [ ]:
#conv block 4
W4_1 = weight([3,3,256,512])
b4_1 = bias([512])
conv4_1 = tf.nn.relu(conv2d(C3_pool, W4_1, b4_1))

W4_2 = weight([3,3,512,512])
b4_2 = bias([512])
conv4_2 = tf.nn.relu(conv2d(conv4_1, W4_2, b4_2))

W4_3 = weight([3,3,512,512])
b4_3 = bias([512])
conv4_3 = tf.nn.relu(conv2d(conv4_2, W4_3, b4_3))

C4_pool = max_pool_2x2(conv4_3)

In [ ]:
#conv block 5
W5_1 = weight([3,3,512,512])
b5_1 = bias([512])
conv5_1 = tf.nn.relu(conv2d(C4_pool, W5_1, b5_1))

W5_2 = weight([3,3,512,512])
b5_2 = bias([512])
conv5_2 = tf.nn.relu(conv2d(conv5_1, W5_2, b5_2))

W5_3 = weight([3,3,512,512])
b5_3 = bias([512])
conv5_3 = tf.nn.relu(conv2d(conv5_2, W5_3, b5_3))

C5_pool = max_pool_2x2(conv5_3)

In [ ]:
#fully connected layers
D_flat = tf.reshape(C5_pool, [-1,512*7*7])

W_D1 = weight([512*7*7, 4096])
b_D1 = bias([4096])
D1_Hidden = tf.nn.relu(tf.matmul(D_flat,W_D1)+b_D1)

W_D2 = weight([4096, 4096])
b_D2 = bias([4096])
D2_Hidden = tf.nn.relu(tf.matmul(D1_Hidden,W_D2)+b_D2)

W_D3 = weight([4096, 3])
b_D3 = bias([3])
y_predict = tf.nn.softmax(tf.matmul(D2_Hidden,W_D3)+b_D3)

In [ ]:
#training method
y_label = tf.placeholder('float', [None, 3])
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predict, labels=y_label))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_function)

In [ ]:
#accuracy Evaluation method
correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_predict, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [ ]:
#start training
trainEpochs = 10
batchSize = 100
totalbatch = int(len(train_images)/batchSize)
loss_list = []; epoch_list = []; accuracy_list = []
from time import time
startTime=time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(trainEpochs):
  for i in range(totalbatch):
    batch_x = train_images[i*batchSize:(i+1)*batchSize]
    batch_y = train_labels[i*batchSize:(i+1)*batchSize]
    sess.run(optimizer, feed_dict={x: batch_x, y_label: batch_y})
  
  loss, acc = sess.run([loss_function, accuracy], feed_dict={x: validation_images, y_label: validation_labels})

  epoch_list.append(epoch)
  loss_list.append(loss)
  accuracy_list.append(acc)
  print("Train Epoch:", '%02d' %(epoch+1), "Loss=", "{:.9f}".format(loss), "Accuracy=", acc)

duration = time() - startTime
print("Training Time", duration)

In [ ]:
#accuracy of testing data 
#print("Accuracy:", sess.run(accuracy, feed_dict={x: test_X, y_label: test_Y_onehot}))
batch_test = 10
totalbatch_test = int(len(test_X)/batch_test)

In [ ]:
t_total_acc = 0
for i in range(totalbatch_test):
    test_batch_x = test_X[i*batch_test:(i+1)*batch_test]
    test_batch_y = test_Y_onehot[i*batch_test:(i+1)*batch_test]

    t_batch_acc = sess.run(accuracy, feed_dict={x: test_batch_x, y_label: test_batch_y})
    t_total_acc = t_total_acc + t_batch_acc

t_total_acc = t_total_acc/totalbatch_test

In [ ]:
print(t_total_acc)

In [ ]:
"""#predict the testing data and see the real label
prediction_result = sess.run(tf.argmax(y_predict,1), feed_dict={x: test_images})
print(prediction_result[:10])
real_num = np.argmax(test_labels, axis=1)
print(real_num[:10])"""